# Import

In [1]:
import os
import sys
from datetime import datetime
import time

import re

import tqdm
import itertools

import json
import urllib.request

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import lxml


# URL Rule

In [ ]:
# comp_id = 380725    # 기업 고유 UID
p_num = 45      # 리뷰 페이지 (4 reviews / page)

### query로 CompID 가져오기

In [ ]:
Keyword = '삼성전자'

In [ ]:
search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
search_req = requests.get(search_url.format(Keyword))
print(search_req.text)

In [ ]:
search_soup = BeautifulSoup(search_req.text, 'lxml')
search_soup.text

In [ ]:
search_soup.select('li:nth-child(1) > div.txt > p.name > a')

In [ ]:
comp_id = re.sub(r'[^0-9]', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))
comp_id

In [ ]:
URL = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=4&isNew=false&employType=1&isEmploy=true&jobCode='
print(URL)

# Json to DataFrame 

In [ ]:
# Json 형식의 웹 데이터 가져오기
raw_data = requests.get(URL).json()
print(raw_data)

In [ ]:
data = raw_data['reviewList']
data

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
df.columns

In [ ]:
df.drop(['CI', 'Gender2', 'Answer', 'UsefulY', 'CareerYearYN',  'MyUsefulY', 'MyOpinion', 'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN', 'Area'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
# 정규식으로 이스케이프 문자 제거        # Good, Bad 리뷰 항목에만 적용.
df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df

In [ ]:
# RegDate 컬럼의 데이터 - 문자열 9번째 까지 출력
df['RegDate'] = df['RegDate'].apply(lambda x: x[:10])
df

# For 문으로 만들어보기
    - 리뷰 데이터가 있는 페이지까지 반복

In [ ]:
rv = []

for p_num in range(1, 52) :
    url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=4&isNew=false&employType=1&isEmploy=true&jobCode='
    rv.append(requests.get(url).json()['reviewList'])


data = list(itertools.chain.from_iterable(rv))
df = pd.DataFrame(data)


df.drop(['CI', 'Gender2', 'Answer', 'UsefulY', 'CareerYearYN',  'MyUsefulY', 'MyOpinion', 'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN', 'Area'], axis=1, inplace=True)
df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:8].strip())


df

# While 문으로 만들기 
    - 그 다음 페이지에 리뷰 데이터 개수가 0 이면 멈춤


In [ ]:
# len(requests.get(url).json()['reviewList'])

In [ ]:
comp_name = '삼성전자'
search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
search_req = requests.get(search_url.format(comp_name)) # Keyword = comp_name
# print(search_req.text)

search_soup = BeautifulSoup(search_req.text, 'lxml')
# search_soup.text
search_soup.select('li:nth-child(1) > div.txt > p.name > a')

comp_id = re.sub(r'[^0-9]', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))
comp_id

In [ ]:

try : 
    # comp_id = 380725
    p_num = 1

    rv = []


    while True : 

        url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=2000&isNew=false&employType=1&isEmploy=false&jobCode='
        rv.append(requests.get(url).json()['reviewList'])

        p_num += 1

        if len(requests.get(url).json()['reviewList']) == 0 :
            break


    data = list(itertools.chain.from_iterable(rv))
    df = pd.DataFrame(data)

    df.drop(['CI', 'Gender2', 'Answer', 'UsefulY', 'CareerYearYN',  'MyUsefulY', 'MyOpinion', 
            'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN', 'Area'], 
            axis=1, inplace=True)
    df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
    df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
    df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:8].strip())
    


except :
    print("리뷰가 존재하지 않습니다.")


df


# 컬럼명 및 데이터 변경

### 컬럼명 변경

In [ ]:
df.columns

In [ ]:
df.rename(columns=
          {'CompName':'comp_name', 
           'RegDate':'review_date', 
           'EmployText':'is_office', 
           'Good':'review_pos', 
           'Bad':'review_neg', 
           'MyStarScore':'review_rate', 
           'JobName':'position'
           }, inplace=True)

df

In [ ]:
df.columns

### 전/현직 여부를 Boolean 값으로 변경

In [ ]:
df['is_office']

In [ ]:
df['is_office'] = df['is_office'].apply(lambda x: x.replace('현직', '1').strip() == '1')

In [ ]:
df['is_office']

# DataFrame to CSV file

In [ ]:
SAVE_PATH = r'./data/'

In [ ]:
df['CompName'][0]

In [ ]:
comp = df['CompName'][0]

In [ ]:
os.makedirs(SAVE_PATH, exist_ok=True)

# csv 파일로 저장.

file_name = f"{comp}_reviews.csv"
save_file_path = os.path.join(SAVE_PATH, file_name)

df.to_csv(save_file_path, index=False, encoding = "utf-8")

# 함수화

In [2]:
SAVE_PATH = r'./data/'

In [3]:
def RV_Catch(comp = str): 
    
    search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
    search_req = requests.get(search_url.format(comp)) # Keyword = comp_name
    # print(search_req.text)

    search_soup = BeautifulSoup(search_req.text, 'lxml')
    # search_soup.text
    search_soup.select('li:nth-child(1) > div.txt > p.name > a')

    comp_id = re.sub(r'[^0-9]', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))





    p_num = 1

    rv = []



    try : 

        while True : 

            url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=2000&isNew=false&employType=1&isEmploy=false&jobCode='
            rv.append(requests.get(url).json()['reviewList'])

            p_num += 1

            if len(requests.get(url).json()['reviewList']) == 0 :
                break


        data = list(itertools.chain.from_iterable(rv))
        df = pd.DataFrame(data)


        df.drop(['CI', 'Gender2', 'Answer', 'UsefulY', 'CareerYearYN',  'MyUsefulY', 'MyOpinion', 
                'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN', 
                'RecomName', 'EmployType', 'NewOld', 'CareerYear', 'Area'], 
                axis=1, inplace=True)
        
        df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
        df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())

        df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:8].strip())

        df['EmployText'] = df['EmployText'].apply(lambda x: x.replace('현직', '1').strip() == '1')



        df.rename(columns=
          {'CompName':'comp_name', 
           'RegDate':'review_date', 
           'EmployText':'is_office', 
           'Good':'review_pos', 
           'Bad':'review_neg', 
           'MyStarScore':'review_rate', 
           'JobName':'position'
           }, inplace=True)

        
        
        
        # csv 파일로 저장.

        comp = df['comp_name'][0]

        os.makedirs(SAVE_PATH, exist_ok=True)

        file_name = f"{comp}_reviews.csv"
        save_file_path = os.path.join(SAVE_PATH, file_name)

        df.to_csv(save_file_path, index=False, encoding = "utf-8")
        


        
    except : 
        print("리뷰가 존재하지 않습니다.")




In [4]:
RV_Catch('SK하이')

In [5]:
pd.read_csv('./data/SK하이닉스_reviews.csv')

,idx,CompID,comp_name,review_date,is_office,review_pos,review_neg,position,review_rate
0,208734,383511,SK하이닉스,20230722,False,어르신들도 알 정도로의 인지도의 대기업. 괜찮은 급여수준.,업무량이 꽤 있는 편이라고 생각합니다. 워라밸 측면에서는 좀 아쉬움,기획/전략/경영,3.2
1,207935,383511,SK하이닉스,20230620,True,"1. 유연근무제 , 해피프라이데이 등 근무 자유로움 2. 상대적으로 높은 연봉 3....",1. 반도체 업계 특성상 사이클을 탐 2. 팀바팀이지만 야근하는 경우도 많음 3. ...,반도체/디스플레이,5.0
2,207626,383511,SK하이닉스,20230611,True,팀바팀 부바부가 심하지만 대체로 제조기술쪽 직군은 삼성전자에 비해 편한편이다. 또 ...,다만 제조기술이 아닌팀 그리고 일부 팀들은 야근이 매우 많은팀들이 있긴하다. 추가로...,생산/제조/설비/조립,3.8
3,207299,383511,SK하이닉스,20230609,True,회사 복지 면이 엄청 좋은 것 같다. 회사가 어려워서 복지 면에서 축소가 되지 않을...,보수적인 상사분들이 있다. (보통 나이가 많으신 분들 중에) 유연근무제이긴 하나 부...,생산/제조/설비/조립,4.2
4,207145,383511,SK하이닉스,20230608,True,급여 수준이 높고 유연 근무제가 시행되어 출퇴근이 자유롭다. 한달에 한번 해피프라이...,서울 경기권에서 지역 마다 다르지만 출퇴근 시간이 오래걸린다. 조직 문화가 다소 경...,생산관리/공정관리/품질관리,4.4
...,...,...,...,...,...,...,...,...,...
81,124584,383511,SK하이닉스,20201118,True,"대기업인만큼 복지가 잘 되어있음, 하루 네끼 무료","연봉 인상률이 낮은 점, 회사 주변 인프라가 별로 좋지 않다",반도체/디스플레이,4.4
82,124452,383511,SK하이닉스,20201118,True,반도체가 호황이라 돈을 많이 주지만 일하는 만큼 받는건지는 모르겠음,모든 남초 회사가 그러하듯 여혐 분위기가 기본적으로 깔려있음,반도체/디스플레이,3.8
83,122620,383511,SK하이닉스,20201117,False,복지 좋음. 식사 시설 등 제공 좋습니다. 만족해요 다양한 선택권,위치 안좋음. 매일 셔틀 지옥입니다 안좋아요. 서울에서 셔틀 타고 다녀야함,반도체/디스플레이,4.0
84,121382,383511,SK하이닉스,20201010,True,복지혜택이 많다. 월급이 많다. 성과급이 많다,지리적으로 멀다. 일부 직무는 매우 비선호 된다,반도체/디스플레이,5.0


In [6]:
RV_Catch('화인석재')

리뷰가 존재하지 않습니다.


In [ ]:
RV_Catch('삼성전자')

In [ ]:
RV_Catch('네이버')

In [ ]:
RV_Catch('카카오')